In [1]:
!nvidia-smi

Tue Jul  2 22:32:29 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 551.23                 Driver Version: 551.23         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2080 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
|  0%   41C    P8              8W /  280W |     312MiB /   8192MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report,accuracy_score,ConfusionMatrixDisplay,confusion_matrix,precision_score,recall_score,roc_curve,roc_auc_score,balanced_accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
import datetime
from dataclasses import dataclass
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from scipy.ndimage import gaussian_filter1d
from itertools import combinations as comb
from pyts.image import RecurrencePlot
import PIL
from sklearn.utils import class_weight
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV,StratifiedKFold
from unidecode import unidecode

In [3]:
X_train_feat_norm_agg = pd.read_csv('X_train_feat_norm_agg.csv')
y_train_feat_norm_agg = pd.read_csv('y_train_feat_norm_agg.csv')
X_test_feat_norm_agg = pd.read_csv('X_test_feat_norm_agg.csv')
y_test_feat_norm_agg = pd.read_csv('y_test_feat_norm_agg.csv')

In [4]:
print(X_train_feat_norm_agg.shape)
print(y_train_feat_norm_agg.shape)
print(X_test_feat_norm_agg.shape)
print(y_test_feat_norm_agg.shape)

(2312, 12)
(2312, 1)
(1810, 12)
(1810, 1)


In [5]:
print(X_train_feat_norm_agg.columns)

Index(['AccX', 'AccY', 'AccZ', 'GyroX', 'GyroY', 'GyroZ', 'AccMagnitude',
       'GyroMagnitude', 'JerkX', 'JerkY', 'JerkZ', 'JerkMagnitude'],
      dtype='object')


In [6]:
# ['AccX', 'AccY', 'AccZ', 'GyroX', 'GyroY', 'GyroZ']

In [7]:
X_train = X_train_feat_norm_agg[['AccX', 'AccY', 'AccZ', 'GyroX', 'GyroY', 'GyroZ']]
X_test = X_test_feat_norm_agg[['AccX', 'AccY', 'AccZ', 'GyroX', 'GyroY', 'GyroZ']]
print(X_train.shape)
print(X_test.shape)

(2312, 6)
(1810, 6)


In [8]:
y_train = y_train_feat_norm_agg
y_test =  y_test_feat_norm_agg

In [9]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(2312, 6)
(2312, 1)
(1810, 6)
(1810, 1)
